### IMPORT

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.applications.nasnet import NASNetMobile

os.environ["CUDA_VISIBLE_DEVICES"]= "1"

tf.debugging.set_log_device_placement(True)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

### LOAD DATA

#### Train Dataset

In [ ]:
real_train_dir = './dataset/train/real/'
real_train = [real_train_dir + x for x in sorted(os.listdir(real_train_dir))]

fake_train_dir = './dataset/train/fake/'
fake_train = [fake_train_dir + x for x in sorted(os.listdir(fake_train_dir))]

#### Test Dataset

In [ ]:
real_test_dir = './dataset/test/real/'
real_test = [real_test_dir + x for x in sorted(os.listdir(real_test_dir))]

fake_test_dir = './dataset/test/fake/'
fake_test = [fake_test_dir + x for x in sorted(os.listdir(fake_test_dir))]

#### Validation Dataset

In [ ]:
real_val_dir = './dataset/val/real/'
real_val = [real_val_dir + x for x in sorted(os.listdir(real_val_dir))]

fake_val_dir = './dataset/val/fake/'
fake_val = [fake_val_dir + x for x in sorted(os.listdir(fake_val_dir))]

#### Train Path

In [ ]:
train_path = real_train + fake_train
train_label = ['0']*len(real_train) + ['1']*len(fake_train)
train_df = pd.DataFrame({'path': train_path, 'label':train_label})
train_df = shuffle(train_df)

train_df.head()

#### Test Path

In [ ]:
test_path = real_test + fake_test
test_label = ['0']*len(real_test) + ['1']*len(fake_test)
test_df = pd.DataFrame({'path': test_path, 'label':test_label})
test_df = shuffle(test_df)

test_df.head()

#### Validation Path

In [ ]:
val_path = real_val + fake_val
val_label = ['0']*len(real_val) + ['1']*len(fake_val)
val_df = pd.DataFrame({'path': val_path, 'label':val_label})
val_df = shuffle(val_df)

val_df.head()

#### Data Augmentation
- rotation_range : 회전 최대 반경
- width_shift_range : 좌우 이동 최대 이미지 가로 사이즈
- height_shift_range : 상하 이동 최대 이미지 세로 사이즈
- horizontal_flip : 좌우 반전 실행
- vertical_flip : 상하 반전 실행
- rescale : 원본은 0-255의 RGB 계수로 구성되는데, 이는 모델을 효과적으로 학습시키기에 너무 높습니다  (통상적인 learning rate를 사용할 경우). 그래서 이를 1/255로 스케일링하여 0-1 범위로 변환시켜줍니다. 

In [ ]:
train_datagen = ImageDataGenerator(rotation_range = 270, width_shift_range = .2,
                                  height_shift_range = .2, horizontal_flip = True,
                                  vertical_flip = True, rescale = 1/255)

test_datagen = ImageDataGenerator(rescale = 1/255)

val_datagen = ImageDataGenerator(rescale = 1/255)

#### Image Read
- flow_from_dataframe() : 디렉토리에서 이미지를 읽을 객체 생성
- train_df : 데이터 프레임
- X_col, y_col : 컬럼 지정
- batch_size : 한번에 리턴 할 이미지의 개수
- class_mode : 분류 방법 (binary : 이진 분류, categorical : 다중 분류)

In [ ]:
IM_HEIGHT = 224
IM_WIDTH = 224

train_generator = train_datagen.flow_from_dataframe(train_df, x_col="path", y_col="label", target_size=(IM_HEIGHT, IM_WIDTH),
                                                   batch_size=16, class_mode='binary', shuffle=True)

test_generator = test_datagen.flow_from_dataframe(test_df, x_col="path", y_col="label", target_size=(IM_HEIGHT, IM_WIDTH),
                                                   batch_size=16, class_mode='binary', shuffle=True)

val_generator = val_datagen.flow_from_dataframe(val_df, x_col="path", y_col="label", target_size=(IM_HEIGHT, IM_WIDTH),
                                                   batch_size=16, class_mode='binary', shuffle=True)

#### Create Model
- NASNetMobile 구조를 갖는 Model 생성

In [ ]:
NASNetMobile = NASNetMobile(input_shape=(IM_HEIGHT, IM_WIDTH, 3),
                     include_top=False, weights='imagenet')

NASNetMobile.summary()

- Sequential() : 입력값을 읽어 예측을 할 Sequential 객체 생성
- add(NASNetMobile) : NASNetMobile 대입
- Flatten() : 선형 회귀를 하기 위해 합성곱 연산을 수행한 결과를 1차원 배열 변환
- Dense : 선형 회귀를 수행할 객체
- Activation = 'relu' : 선형 회귀 후 relu 활성 함수 사용
- model.add(Dense(512, activation="relu")) : Dense 모델을 예측 할 수 있도록 model에 추가
- Dropout : Model Overfitting 방지
- Dense(1) : 출력 데이터 칸의 수는 1
- Activation = 'sigmoid' : 선형 회귀 후 sigmoid 함수를 활성 함수를 이용해 0, 1 값 리턴
- loss : 손실 함수는 모델을 컴파일하기 위해 필요한 변수 중 하나
- binary_crossentropy : 이항 교차 엔트로피(두 개의 클래스 중에서 예측할 때)
- optimizer=Adam(lr=.0001) : learning_rate를 0.0001로 설정
- model_path : 모델 저장 경로 지정
- EarlyStopping : 모델 학습 사전 중지 (monitor에서 정해준 기준이 개선 안될 시 중지)
- monitor : val_loss를 모니터링, 사전 중지 기준
- patience : 횟수를 주면 개선이 안돼도 횟수만큼 더 학습

In [ ]:
model = Sequential()
model.add(NASNetMobile)
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy', optimizer=Adam(lr=.0001), metrics=['acc'])

model_path ="./model/NASNetMobile-{epoch:02d}-{val_loss:.4f}.h5"
checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=30)

- summary : 생성된 모델 정보 출력

In [ ]:
model.summary()

#### Model learn
- fit_generator : 제너레이터로 이미지를 담고 있는 배치로 학습
- train_generator : 학습 데이터
- epochs : 학습 할 횟수
- validation_data : 학습 검증 데이터

In [ ]:
history = model.fit_generator(train_generator, epochs=5000, validation_data = val_generator, callbacks=[checkpoint, early_stopping])

#### Visualize the learning process

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss', 'acc', 'val_acc'], loc='upper left')
plt.show()